<a href="https://colab.research.google.com/github/16520124/CS114.K11/blob/master/MaskDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#final
import cv2 #import các thư viện 
import os
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

In [0]:
#final
from google.colab import drive #kết nối với gg drive
drive.mount('/content/drive')

In [0]:
cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [0]:
#final
def Get_first_face(image_path): #trả về khuôn mặt đầu tiên từ một bức ảnh và số lượng face trong 1 ảnh
  img = cv2.imread(image_path)
  cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  faces = cascade.detectMultiScale(img)
  print('found', len(faces), 'faces', sep=' ')
  if len(faces) == 0: #không tìm thấy face nào trong ảnh
    return None, 0

  (x, y, w, h) = faces[0] #giả sử detect nhiều face thì lấy face đầu tiên 
  r = max(w, h) / 2
  centerx = x + w / 2
  centery = y + h / 2
  nx = int(centerx - r)
  ny = int(centery - r)
  nr = int(r * 2)

  faceimg = img[ny:ny+nr, nx:nx+nr]
  #faceimg = cv2.resize(faceimg,(50,50))
  #faceimg = cv2.cvtColor(faceimg, cv2.COLOR_BGR2GRAY)   #để dành làm ở bước sau
  return faceimg, len(faces) #độ dài mảng faces -> số lượng face detect


In [0]:
def Resize_image(faceimg): #resize ảnh (face) về kích thước 50x50 pixels, chuyển về màu xám
  faceimg=cv2.resize(faceimg, (50,50))
  faceimg=cv2.cvtColor(faceimg, cv2.COLOR_BGR2GRAY)
  return faceimg


In [0]:
#Tạo 2 pandas series X và Y từ thư mục chứa ảnh train
#Duyệt từng file trong thư mục, crop ảnh, resize ảnh crop về kích thước 50x50 và đổi sang ảnh xám. Sau đó flatten, và đưa vào X series. 
#Nếu ảnh không crop được cái face nào thì cho mặc định một face toàn màu đen. 
#Series Y được xác định dựa vào tên file ảnh. Tên bắt đầu bằng chữ 'pos' là có đeo khẩu trang, 'neg' là không đeo khẩu trang.

i = 1; #biến đếm i để chặn số lượng ảnh duyệt
#folder = '/mydrive/My Drive/Teaching/2019.hk2.cs114.mask.dataset/train'
work_folder = '/content/drive/My Drive/train'

all_files = os.listdir(work_folder)

#count = 1 -> i
face_amount = 0
dectect = 0
faces_in_image = 0

X = pd.DataFrame(columns=range(2500)) #50x50=2500
Y = pd.DataFrame(columns=['label']) #gán nhãn dựa vào tên bắt đầu của ảnh

for image in os.listdir(work_folder):
  print('====  process image', image, ':', i, 'out of', len(all_files), sep=' ')
  faceimg,faces_in_image = Get_first_face(work_folder + "/" + image)
  if faceimg is not None:
    #new_img = cv2.cvtColor(cv2.resize(faceimg,(50,50)), cv2.COLOR_BGR2GRAY)
    new_img=Resize_image(faceimg)

    flat = new_img.flatten()
    t = pd.DataFrame([flat])
    dectect += 1
    #print(t)

  else:
    print('Cant find face')
    t = pd.DataFrame([[0]*2500], columns=range(2500)) #màu đen

  X=X.append(t, ignore_index=True)

  Y=Y.append({'label':image[0:3]},ignore_index=True) #nhãn là pos hoặc neg

  i = i + 1
  """if i == 15: #chỉ duyệt qua 14 ảnh
    break;"""
print(X)
print(Y)

In [0]:
Transform_to_bool = LabelEncoder()
labels = Transform_to_bool.fit_transform(Y) #chuyển nhãn pos thành 1, neg thành 0 để tiện xử lý
labels

In [0]:
#chọn model để học 
#chọn kiến trúc mạng CNN vgg16
#base_model = VGG16(weights = 'imagenet', include_top = False)
base_model = VGG16(include_top= False, weights='imagenet' , input_shape = (50,50,3))

In [0]:
base_model.summary()

In [0]:
list_image = []

In [0]:
#trích xuất các features
features = model.predict(list_image)
# Chuyển tensor thành vector, convolution layer cuối (512*7*7)
features = features.reshape((features.shape[0], 512*7*7))

In [0]:
#chia tập dữ liệu ban đầu thành training_set và test_set với tỷ lệ 85:15

In [0]:
#đánh giá model

In [0]:
#Choose model and learn
from sklearn.svm import SVC

model = SVC()
model.fit( X,  Y.values.ravel()  )

print(model)



In [0]:
#dùng pickle để lưu lại model
import pickle

string = pickle.dumps(model)
print(string)


In [0]:
def show_metrics(X, Y, string):
  import pickle
  print(Y.values.ravel())

  loadmodel = pickle.loads(string)
  pred = loadmodel.predict(X)
  print(pred)
  from sklearn import metrics
  print("Acurracy ", metrics.accuracy_score(Y.values.ravel(), pred))
  print("Precision ", metrics.precision_score(Y.values.ravel(), pred, pos_label='pos'))
  print("Recall ", metrics.recall_score(Y.values.ravel(), pred, pos_label='pos'))
  print("F1 ", metrics.f1_score(Y.values.ravel(), pred, pos_label='pos'))
  #print("Acurracy ", metrics.accuracy_score(Y_test.values.ravel(), pred))